In [2]:
%reload_ext autoreload
%autoreload

%reload_ext autoreload
%autoreload 2
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
from tqdm import tqdm

# Increase the resolution of all the plots below
plt.rcParams.update({"figure.dpi": 200,"figure.facecolor":"w","figure.figsize": (15,10)})

# import helper functions
from src.utls import compute_repeated_stats

In [3]:
# (import) helper functions
dir_proj = Path("__file__").absolute().parents[1]
dir_rawdata = Path.joinpath(dir_proj, "data")
dir_prep = Path.joinpath(dir_proj,"results")
print(dir_rawdata)


c:\Users\User\Desktop\kiel\tremor_music_ap\data


In [26]:
# load data
pd.read_csv(Path.joinpath(dir_prep,"single_subj_pow.csv"))
peaks = pd.read_csv(Path.joinpath(dir_prep,"single_subj_agg.csv"))
group_asgn = pd.read_excel(Path.joinpath(dir_rawdata,"group_assign.xlsx"), index_col=None)

In [178]:
# define cfg vars
cfg_srate = 100
cfg_group = {1:"Standard & exp. music", 2:"Standard & unspecific music", 3: "Only Standard", 4: "Placebo"}
cfg_trial = {1: "Standing flexed", 2:"Sitting", 3: "Sitting relaxed"}
cfg_visit = {1:"Inital", 2:"14-days"}

idx_axis_oi = peaks["axis"] == "y"
idx_task_oi = peaks["task"] == "Sitting flexed"
idx_visit_oi = peaks["visit"] != "Follow-up"
peaks.head()

,id,visit,task,axis,peak frequency,peak amplitude
0,BS0068,Inital,Standing flexed,x,5.865385,9544.121503
1,BS0068,Inital,Standing flexed,y,5.865385,373.978677
2,BS0068,Inital,Standing flexed,z,5.865385,162159.434519
3,BS0068,Inital,Sitting flexed,x,5.288462,3445.162849
4,BS0068,Inital,Sitting flexed,y,5.000000,450.935236


In [37]:
# assign group
group = pd.melt(group_asgn, value_name="id").dropna()
peaks_group = pd.merge(peaks, group, on="id", how="outer")

idx_visit_oi = peaks_group["visit"] != "Follow-up"
data_oi = peaks_group[idx_visit_oi]
data_oi = data_oi.groupby('id').filter(lambda x: {"Inital","14-days"}.issubset(x['visit']))
data_oi = data_oi.rename(columns={"variable": "group"})
sumed_axis = data_oi.groupby(['id','visit','group','task'],as_index=False)[["peak amplitude","peak frequency"]].sum()

n_rows = sumed_axis.shape[0]
sumed_axis["Norm amp"] = np.nan
sumed_axis["Norm freq"] = np.nan

for i in range(0,n_rows):
    idx_id = sumed_axis["id"].str.match(sumed_axis["id"][i])
    sumed_axis["Norm amp"][i] = sumed_axis["peak amplitude"][i] / np.sum(sumed_axis["peak amplitude"][idx_id])
    sumed_axis["Norm freq"][i] = sumed_axis["peak frequency"][i] / np.sum(sumed_axis["peak frequency"][idx_id])
    
sumed_axis.to_csv(Path.joinpath(dir_prep,"group_anova.csv"))

C:\Users\User\AppData\Local\Temp\ipykernel_12924\3338683340.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumed_axis["Norm amp"][i] = sumed_axis["peak amplitude"][i] / np.sum(sumed_axis["peak amplitude"][idx_id])
C:\Users\User\AppData\Local\Temp\ipykernel_12924\3338683340.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumed_axis["Norm freq"][i] = sumed_axis["peak frequency"][i] / np.sum(sumed_axis["peak frequency"][idx_id])


In [28]:
t_amp,p_amp, t_freq, p_freq = compute_repeated_stats(peaks,"Sitting releaxed")
print(f"Diff amp is {p_amp:.3f}; Diff freq is {p_freq:.3f};")

Diff amp is nan; Diff freq is nan;
